In [2]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

In [3]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_2_Medoid_Cluster_1_pvalue_02.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [4]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))


In [5]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'


In [6]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp



In [7]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)



In [8]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:
    print(pair)
    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"


file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_2_Medoids_Cluster_1_pValue_02.txt', 'w')
file.write(pos_result)
file.close()



['A', 'EW']
['AAPL', 'ANTM']
['AAPL', 'AVGO']
['AAPL', 'BAX']
['AAPL', 'BIIB']
['AAPL', 'CHTR']
['AAPL', 'COP']
['AAPL', 'CSX']
['AAPL', 'FDX']
['AAPL', 'GILD']
['AAPL', 'MS']
['AAPL', 'RMD']
['AAPL', 'SPGI']
['AAPL', 'TWX']
['ABBV', 'AET']
['ABBV', 'ANTM']
['ABBV', 'ATVI']
['ABBV', 'AVGO']
['ABBV', 'CSX']
['ABBV', 'FDX']
['ABBV', 'GD']
['ABBV', 'GILD']
['ABBV', 'GS']
['ABBV', 'MS']
['ABBV', 'NVDA']
['ABBV', 'SCHW']
['ABBV', 'TWX']
['ABBV', 'WBA']
['ABT', 'AET']
['ABT', 'ANTM']
['ABT', 'AVGO']
['ABT', 'BK']
['ABT', 'BRKB']
['ABT', 'C']
['ABT', 'CHTR']
['ABT', 'CMCSA']
['ABT', 'CME']
['ABT', 'CSCO']
['ABT', 'CVS']
['ABT', 'DIS']
['ABT', 'EL']
['ABT', 'EOG']
['ABT', 'IBM']
['ABT', 'JPM']
['ABT', 'KO']
['ABT', 'MCD']
['ABT', 'MON']
['ABT', 'MS']
['ABT', 'NEE']
['ABT', 'NVDA']
['ABT', 'OXY']
['ABT', 'PG']
['ABT', 'PM']
['ABT', 'PNC']
['ABT', 'SCHW']
['ABT', 'T']
['ABT', 'TWX']
['ABT', 'UNH']
['ABT', 'USB']
['ABT', 'VZ']
['ABT', 'WMT']
['ABT', 'XOM']
['ACN', 'AET']
['ACN', 'AMT']
['ACN', 'A

['BDX', 'PFE']
['BDX', 'PG']
['BDX', 'PNC']
['BDX', 'SCHW']
['BDX', 'SLB']
['BDX', 'TXN']
['BDX', 'USB']
['BDX', 'UTX']
['BDX', 'VAR']
['BDX', 'WMT']
['BDX', 'XOM']
['BDX', 'ZBH']
['BK', 'BRKB']
['BK', 'CAT']
['BK', 'CB']
['BK', 'CHTR']
['BK', 'CL']
['BK', 'CMCSA']
['BK', 'CME']
['BK', 'COST']
['BK', 'CSCO']
['BK', 'CSX']
['BK', 'CVX']
['BK', 'DIS']
['BK', 'EL']
['BK', 'GD']
['BK', 'HOLX']
['BK', 'HON']
['BK', 'IBM']
['BK', 'IDXX']
['BK', 'ISRG']
['BK', 'JNJ']
['BK', 'KO']
['BK', 'LLY']
['BK', 'MCD']
['BK', 'MDLZ']
['BK', 'MDT']
['BK', 'MO']
['BK', 'MON']
['BK', 'MRK']
['BK', 'NEE']
['BK', 'OXY']
['BK', 'PEP']
['BK', 'PFE']
['BK', 'PG']
['BK', 'PM']
['BK', 'PNC']
['BK', 'QCOM']
['BK', 'RMD']
['BK', 'SCHW']
['BK', 'SYK']
['BK', 'T']
['BK', 'TJX']
['BK', 'TWX']
['BK', 'UNP']
['BK', 'UPS']
['BK', 'USB']
['BK', 'UTX']
['BK', 'VAR']
['BK', 'VZ']
['BK', 'WFC']
['BK', 'WMT']
['BK', 'XOM']
['BK', 'ZBH']
['BLK', 'CME']
['BLK', 'COST']
['BLK', 'CSX']
['BLK', 'GM']
['BLK', 'MDT']
['BLK', 'MMM']
[

['JNJ', 'KO']
['JNJ', 'MSFT']
['JNJ', 'OXY']
['JNJ', 'PG']
['JNJ', 'SCHW']
['JNJ', 'XOM']
['JPM', 'KO']
['JPM', 'LLY']
['JPM', 'MCD']
['JPM', 'MO']
['JPM', 'MON']
['JPM', 'MRK']
['JPM', 'NEE']
['JPM', 'NKE']
['JPM', 'OXY']
['JPM', 'PEP']
['JPM', 'PFE']
['JPM', 'PG']
['JPM', 'PKI']
['JPM', 'PM']
['JPM', 'PNC']
['JPM', 'QCOM']
['JPM', 'RMD']
['JPM', 'SCHW']
['JPM', 'SYK']
['JPM', 'T']
['JPM', 'TJX']
['JPM', 'TMO']
['JPM', 'TWX']
['JPM', 'UNP']
['JPM', 'UPS']
['JPM', 'VAR']
['JPM', 'VZ']
['JPM', 'WMT']
['JPM', 'XOM']
['JPM', 'ZBH']
['KO', 'LMT']
['KO', 'MCD']
['KO', 'MMM']
['KO', 'MO']
['KO', 'MSFT']
['KO', 'NEE']
['KO', 'NOC']
['KO', 'PEP']
['KO', 'PFE']
['KO', 'PKI']
['KO', 'PM']
['KO', 'PNC']
['KO', 'QCOM']
['KO', 'SBUX']
['KO', 'SCHW']
['KO', 'SLB']
['KO', 'SPGI']
['KO', 'SYK']
['KO', 'T']
['KO', 'TJX']
['KO', 'TMO']
['KO', 'TWX']
['KO', 'UNH']
['KO', 'UTX']
['KO', 'VAR']
['KO', 'VZ']
['KO', 'WBA']
['KO', 'WFC']
['KO', 'XOM']
['KO', 'ZBH']
['LLY', 'LOW']
['LLY', 'MCD']
['LLY', 'MDLZ']

In [9]:

listOfPosResult = pos_result.split('\n\n')
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

OXY QCOM 1.19
746
1486
Percentage of profiting pairs in Cluster: 50.20188425302826%


In [10]:
listRet.sort()
print(listRet)


[2.4403274038320434e-05, 0.0005789240545618668, 0.0008361256393108363, 0.0018689611613713453, 0.001917605141752574, 0.0019218623264592339, 0.0022196228150706074, 0.002544800705651511, 0.0031318922949384385, 0.0033161124334600343, 0.0036693462703600234, 0.0037845720351992896, 0.0038750611254691593, 0.00389155025505886, 0.004704729639042995, 0.004917345543901632, 0.00520972954128851, 0.005216202981549571, 0.006240359512787279, 0.006610564654159674, 0.0066622146461942575, 0.006904745284947751, 0.007221765975897654, 0.0072729819162032075, 0.007387624516108388, 0.007770828866163697, 0.008013076268423448, 0.008190740229226492, 0.008595349232315214, 0.009127793145639487, 0.009395771110333583, 0.009446671389486311, 0.009568905990733612, 0.009610863181696344, 0.010178347135375106, 0.010613918714477534, 0.010963873470938097, 0.010965863937131956, 0.010989833208752771, 0.01112457996529642, 0.011385585686922326, 0.011886963108794078, 0.012249002959485153, 0.012390230713694592, 0.012731546442784064